In [1]:
!pip install bitsandbytes transformers accelerate codecarbon

In [2]:
!git clone https://github.com/muqtasid87/benchmark_vlm

fatal: destination path 'benchmark_vlm' already exists and is not an empty directory.


In [5]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
from transformers.image_utils import load_image
import torch
import csv
import platform
from transformers import pipeline
import psutil
import numpy as np
from PIL import Image
import os
import shutil
import time
from codecarbon import EmissionsTracker

In [6]:
# Load model and processor
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model_id = "llava-hf/llava-1.5-7b-hf"
pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 4957.88 MB. The target location /root/.cache/huggingface/hub/models--llava-hf--llava-1.5-7b-hf/blobs only has 3340.85 MB free disk space.
  warnings.warn(


model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 4957.88 MB. The target location /root/.cache/huggingface/hub/models--llava-hf--llava-1.5-7b-hf/blobs only has 1.89 MB free disk space.
  warnings.warn(


model-00002-of-00003.safetensors:  67%|######7   | 3.34G/4.96G [00:00<?, ?B/s]

ValueError: Could not load model llava-hf/llava-1.5-7b-hf with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForVision2Seq'>, <class 'transformers.models.llava.modeling_llava.LlavaForConditionalGeneration'>). See the original errors:

while loading with AutoModelForVision2Seq, an error is thrown:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/transformers/modeling_utils.py", line 3990, in from_pretrained
    resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(
      

In [ ]:
# Image directory
img_dir = 'benchmark_vlm/master'
use_axles = False

In [ ]:
wrong_predictions_dir = f'experiment_data/{model_id}/wrong_predictions'  # Directory to store wrongly predicted images
os.makedirs(wrong_predictions_dir, exist_ok=True)

### Inference Function

In [ ]:
# Function to run inference
def infer_image(image_path, prompt):
  conversation = [
      {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image"},
          ],
      },
  ]
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

  with open(image_path, "rb") as f:
      input_image = Image.open(f).convert("RGB")

      outputs = pipe(input_image, prompt=prompt, generate_kwargs={"max_new_tokens": 2})
      outputs = outputs[0]["generated_text"]
      outputs = outputs.split("ASSISTANT: ", 1)[-1]

  return outputs.lower() #outputs only vlm response token in string format

### Validation function

In [ ]:
# Define vehicle classes for validation
vehicle_classes = {
    "Class 1 - Motorcycles": ["motorcycle", "motorbike", "bike", "scooter", "motor", "moped"],
    "Class 2 - Car, SUV, Station Wagons": ["car", "sedan", "hatchback", "suv", "mpv", "crossover", "station wagon", "wagon", "coupe"],
    "Class 3 - 2 axle lorry, van, pickup": ["pickup", "van", "minivan"],
    "Class 4 - 2 axles, 5 or 6 wheels bus or truck": ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 5 - 3 axle truck, bus": ["bus", "truck"],
    "Class 6 - 4 axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 7 - 5 or more axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "bus 2" : ["coach", "bus"],
    "bus 3" : ["coach", "bus"],
    "truck 2" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 3" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 4" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 5" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "van" : ["van", "minivan", 'minibus'],
    "pickup" : ["pickup", "pickup truck", "camper"],
}

valid_vehicle_types = {type for sublist in vehicle_classes.values() for type in sublist}

In [ ]:
# Validate predictions

def validate_prediction(folder_name, prediction, axles=None):
    # Extract folder label
    for class_name, valid_labels in vehicle_classes.items():
        if folder_name.startswith(class_name):
            if axles:  # Use axle count for bus/truck
                if axles == 1 or 0:
                  return False
                elif axles == 2:
                    return "2" in folder_name[-1]
                elif axles == 3:
                    return "3" in folder_name[-1]
                elif axles == 4:
                    return "4" in folder_name[-1]
                elif axles >= 5:
                    return "5" in folder_name[-1]
            if isinstance(valid_labels, list):
                return prediction in valid_labels
            return prediction == valid_labels
    return False

# Main Loop

In [ ]:
from tqdm import tqdm

In [ ]:
# Main processing loop
total_files = sum(len(files) for _, _, files in os.walk(img_dir))
progress_bar = tqdm(total=total_files, desc="Processing Images", unit="file")
start_time = time.time()
tracker = EmissionsTracker()
tracker.start()
results = []
for root, dirs, files in os.walk(img_dir):
    for file in files:
        if file.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(root, file)
            folder_name = os.path.basename(os.path.dirname(image_path))
            progress_bar.set_description(f"Processing: {folder_name}")
            progress_bar.update(1)  # Increment the progress bar
            print(f"Processing: {folder_name}")

            try:
                # First inference: vehicle type
                vehicle_prompt = f"What type of vehicle is this?  Answer in one word"
                # vehicle_prompt = f"What type of vehicle is this? Choose one from: {', '.join(valid_vehicle_types)}. Answer in one word"
                prediction = infer_image(image_path, vehicle_prompt)
                print(f"Vehicle Prediction: {prediction}")

                # Handle buses and trucks
                axles = None
                if prediction in ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck',  "tow truck", "dump trailer"]:
                    # Check if it's a pickup truck
                    if prediction == 'truck':
                        pickup_prompt = "Is this a pickup truck? Respond with only yes or no."
                        is_pickup = infer_image(image_path, pickup_prompt).strip().lower()
                        print(f"Is Pickup Truck: {is_pickup}")
                        if is_pickup == "yes":
                            prediction = "pickup truck"
                        elif use_axles:
                            # Predict axles for non-pickup trucks
                            axle_prompt = "How many axles does this vehicle have? Output a single integer value."
                            axle_prediction = infer_image(image_path, axle_prompt)
                            print(f"Axle Prediction: {axle_prediction}")

                            try:
                                axles = int(axle_prediction)
                            except ValueError:
                                print(f"Invalid axle prediction: {axle_prediction}")
                                axles = None  # Fallback if parsing fails
                    elif use_axles:

                      axle_prompt = "How many axles does this vehicle have? Output a single integer value"
                      axle_prediction = infer_image(image_path, axle_prompt)
                      print(f"Axle Prediction: {axle_prediction}")

                      try:
                          axles = int(axle_prediction)
                      except ValueError:
                          print(f"Invalid axle prediction: {axle_prediction}")

                # Validate prediction
                is_correct = validate_prediction(folder_name, prediction, axles)
                print(f"Validation Result: {is_correct}")

                # Store results
                results.append({
                    "image": image_path,
                    "folder_label": folder_name,
                    "prediction": prediction,
                    "axles": axles,
                    "is_correct": is_correct
                })

                # Handle wrong predictions
                if not is_correct:
                    # Create a folder for the wrong prediction
                    wrong_folder = os.path.join(wrong_predictions_dir, prediction, str(axles))
                    os.makedirs(wrong_folder, exist_ok=True)

                    # Copy the image to the folder
                    shutil.copy(image_path, os.path.join(wrong_folder, file))
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
                continue
emissions = tracker.stop()
# Stop the timer
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total processing time: {elapsed_time:.2f} seconds")
print(f"CO2 emissions: {emissions:.6f} kg")

In [ ]:
# Save results to CSV
output_csv = f'experiment_data/{model_id}/results.csv'
with open(output_csv, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["image", "folder_label", "prediction", "axles", "is_correct"])
    writer.writeheader()
    writer.writerows(results)

# Metrics calculation
correct_predictions = sum(1 for r in results if r["is_correct"])
total_images = len(results)
accuracy = correct_predictions / total_images * 100 if total_images > 0 else 0
wrong_predictions = total_images - correct_predictions

# Per-class metrics
class_metrics = {}
for result in results:
    label = result["folder_label"]
    correct = result["is_correct"]
    if label not in class_metrics:
        class_metrics[label] = {"correct": 0, "total": 0}
    class_metrics[label]["total"] += 1
    if correct:
        class_metrics[label]["correct"] += 1

# Aggregate class metrics
per_class_accuracy = {
    label: (data["correct"] / data["total"]) * 100
    for label, data in class_metrics.items()
}
per_class_error_rate = {
    label: 100 - acc for label, acc in per_class_accuracy.items()
}

# Collect system configuration
system_config = {
    "Processor": platform.processor(),
    "CPU Count": psutil.cpu_count(logical=True),
    "Memory (GB)": round(psutil.virtual_memory().total / (1024 ** 3), 2),
    "Elapsed Time (seconds)": elapsed_time,
    "CO2 Emissions (kg)": round(emissions, 6)
}

# Metrics dictionary
metrics = {
    "Total Images": total_images,
    "Correct Predictions": correct_predictions,
    "Wrong Predictions": wrong_predictions,
    "Accuracy (%)": accuracy,
    "Error Rate (%)": 100 - accuracy
}

# Save metrics to CSV
metrics_csv =  f'experiment_data/{model_id}/metrics.csv'
with open(metrics_csv, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "Metric", "Value"])
    for key, value in metrics.items():
        writer.writerow([model_id, key, value])
    for label, acc in per_class_accuracy.items():
        writer.writerow([model_id, f"Per-Class Accuracy: {label}", acc])
        writer.writerow([model_id, f"Per-Class Error Rate: {label}", per_class_error_rate[label]])
    for key, value in system_config.items():
        writer.writerow([model_id, key, value])

# Print final metrics
print("\n--- Metrics ---")
for key, value in metrics.items():
    print(f"{key}: {value}")
for label, acc in per_class_accuracy.items():
    print(f"Class '{label}' - Accuracy: {acc:.2f}%, Error Rate: {per_class_error_rate[label]:.2f}%")
for key, value in system_config.items():
    print(f"{key}: {value}")